In [ ]:
!cp drive/MyDrive/CS298/malware_data.zip .
!unzip malware_data.zip
!rm malware_data.zip
!rm -rf /content/__MACOSX

In [ ]:
!cp drive/MyDrive/CS298/data.py .
!cp drive/MyDrive/CS298/opdictWinWebSec.json .
!cp drive/MyDrive/CS298/opdictZbot.json .
!cp drive/MyDrive/CS298/opdictVBInject.json .
!cp drive/MyDrive/CS298/opdictOnLineGames.json .
!cp drive/MyDrive/CS298/opdictRenos.json .

In [ ]:
import data

import glob
from __future__ import print_function, division

from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Conv1D
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import plot_model
import tensorflow.keras.backend as kb

import matplotlib.pyplot as plt

import numpy as np

In [ ]:
class WGAN():
    def __init__(self, family, alpha, n_critic, clip_value):
        """
        @params:
        family: malware family
        alpha: learning rate fro RMSProp
        n_critic: number of critic iterations per generator iteration
        clip_value: clipping range
        """
        self.opcode_len = 600
        self.sample_shape = (self.opcode_len,)
        self.latent_dim = 100
        self.family = family

        self.alpha = alpha
        self.n_critic = n_critic
        self.clip_value = clip_value
        optimizer = RMSprop(lr=self.alpha)
        
        print('N_critic: %d, Alpha: %f, Clip_value: %f'%(self.n_critic, self.alpha, self.clip_value))

        ####################################
        """                                
        Critic graph and compilation
        """
        ####################################
        self.critic = self.build_critic()
        self.critic.compile(loss=self.wasserstein_loss,
            optimizer=optimizer,
            metrics=['accuracy'])
        
        ####################################
        """                                
        Generator graph and compilation
        """
        ####################################

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates malware samples
        z = Input(shape=(self.latent_dim,))
        fake_sample = self.generator(z)

        # For the combined model we will only train the generator
        self.critic.trainable = False

        # The critic takes generated samples as input and returns score
        validity = self.critic(fake_sample)

        # The combined model
        # Trains the generator to fool the critic
        self.combined = Model(z, validity)
        self.combined.compile(loss=self.wasserstein_loss, optimizer=optimizer, metrics=['accuracy'])
        
    def wasserstein_loss(self, y_true, y_pred):
        return kb.mean(y_true * y_pred)
    
    def build_generator(self):

        model = Sequential()
 
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2), input_dim=self.latent_dim))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.sample_shape), activation='tanh'))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        output = model(noise)

        plot_model(model, to_file='wgan_generator_architecture.png', show_shapes=True)

        return Model(noise, output)

    def build_critic(self):

        model = Sequential()

        model.add(Dense(512, activation=LeakyReLU(alpha=0.2), input_shape=self.sample_shape))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512, activation=LeakyReLU(alpha=0.2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1))
        model.summary()

        inp = Input(shape=self.sample_shape)
        validity = model(inp)

        plot_model(model, to_file='wgan_critic_architecture.png', show_shapes=True)

        return Model(inp, validity)

    def train(self, X_train, num_unique, epochs, batch_size=32, model_interval=50):
        factor = num_unique/2
        print('Num unique: %d, Factor: %f'%(num_unique, factor))

        # Load the dataset and rescale to [-1, 1]
        X_train = X_train/factor - 1.
        print(X_train.shape)
        
        # Save losses for each epoch for graphs
        results = {}
        
        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake = np.ones((batch_size, 1))

        for epoch in range(epochs):
            if epoch%400 == 0:
                self.n_critic = 100
            else:
                self.n_critic = 10
            for _ in range(self.n_critic):
                # ---------------------
            	#  Train Critic
                # ---------------------

                # Select a random batch of real samples
                idx = np.random.randint(0, X_train.shape[0], batch_size)
                real = X_train[idx]
                
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new samples
                gen = self.generator.predict(noise)

                # Train the critic
                d_loss_real = self.critic.train_on_batch(real, valid)
                d_loss_fake = self.critic.train_on_batch(gen, fake)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)

                # Clip critic weights
                for l in self.critic.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                    l.set_weights(weights)
            
            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the critic label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)
            results[epoch] = [1-d_loss[0], 1-g_loss[0]]
            
            # Print progress
            print ("%d [D loss: %f] [G loss: %f]" % (epoch, 1 - d_loss[0], 1 - g_loss[0]))
            
            if epoch % model_interval == 0:
                self.save_model(epoch)
                
        print('%d Epochs finished. Saving model in Google Drive.' % (epoch))
        return self.plot_losses(results, epochs)
        
    def save_model(self, epoch):
        path = '/content/drive/MyDrive/CS298/WGAN_Results/Models/%s/%s_generator_%d.hdf5' % (self.family, self.family, epoch)
        self.generator.save(path, include_optimizer=True, save_format='h5')
    
    def plot_losses(self, results, epochs):
        print('Plotting.')
        f_d, ax_d = plt.subplots(figsize=(9,6))
        f_g, ax_g = plt.subplots(figsize=(9,6))
        x = [val for val in range(epochs)]
        d = [losses[0] for _,losses in results.items()]
        g = [losses[1] for _,losses in results.items()]
        ax_d.plot(x, d, 'r', label='Critic loss')
        ax_d.set_title('WGAN Critic Loss')
        ax_g.plot(x, g, 'b', label='Generator loss')
        ax_g.set_title('WGAN Generator Loss')
        return f_d, ax_d, f_g, ax_g

In [ ]:
files = glob.glob('Renos/*.txt')
print(len(files))

1568


In [ ]:
#family = ['WinWebSec', 'Zbot', 'VBInject', 'OnLineGames', 'Renos']
family = ['VBInject']
for fam in family:
  print('Current family: %s' % (fam))
  data_obj = data.DATA(fam, 600)
  data_samples, num_unique = data_obj.load_data(True)
  gan = WGAN(fam, 0.0001, 10, 0.5)
  f_d, ax_d, f_g, ax_g = gan.train(data_samples, num_unique, epochs=100001, batch_size=32, model_interval=500)
  plt.legend(loc='lower right')
  f_d.savefig('/content/drive/MyDrive/CS298/WGAN_Results/Losses/wgan_%s_critic.png'%(fam))
  f_g.savefig('/content/drive/MyDrive/CS298/WGAN_Results/Losses/wgan_%s_generator.png'%(fam))
  plt.show()

Current family: VBInject
Total samples for VBInject: 2694
Mappings in JSON file: 24
WGAN
N_critic: 10, Alpha: 0.000100, Clip_value: 0.500000
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               307712    
_________________________________________________________________
batch_normalization_8 (Batch (None, 512)               2048      
_________________________________________________________________
dense_11 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_9 (Batch (None, 512)               2048      
_________________________________________________________________
dense_12 (Dense)             (None, 512)               262656    
_________________________________________________________________
batch_normalization_10 (Batc (None, 512)     

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
print(device_name)

In [ ]:

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

data = ["I love machine learning. Its awesome.",
        "I love coding in python",
        "I love building chatbots",
        "they chat amagingly well"]

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]
print(tagged_data)

[TaggedDocument(words=['i', 'love', 'machine', 'learning', '.', 'its', 'awesome', '.'], tags=['0']), TaggedDocument(words=['i', 'love', 'coding', 'in', 'python'], tags=['1']), TaggedDocument(words=['i', 'love', 'building', 'chatbots'], tags=['2']), TaggedDocument(words=['they', 'chat', 'amagingly', 'well'], tags=['3'])]


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True